# Getting started with Cirq and IonQ on Azure Quantum

This notebooks shows how to send a basic quantum circuit to an IonQ target via Azure Quantum.

First, install `azure-quantum` with the Cirq dependencies:

In [1]:
!pip install azure-quantum[cirq]

Looking in indexes: https://pypi.org/simple, https://pkgs.dev.azure.com/ms-quantum-public/9af4e09e-a436-4aca-9559-2094cfe8d80c/_packaging/alpha/pypi/simple/
ERROR: Could not find a version that satisfies the requirement azure-quantum[cirq]==0.18.2109.xxxxxx (from versions: 0.11.2004.2825, 0.11.2006.207, 0.11.2006.403, 0.12.2007.114, 0.12.2007.124, 0.12.2007.1802, 0.12.2007.2031, 0.12.2008.2513, 0.12.2009.2803, 0.12.2010.301, 0.12.2010.504, 0.13.2010.2604, 0.13.2011.1004, 0.13.2011.119705b1, 0.14.2011.119794b1, 0.14.2011.119823b1, 0.14.2011.120152b1, 0.14.2011.120240, 0.15.2101.22645, 0.15.2101.125897, 0.15.2101.126940, 0.15.2101.126941, 0.15.2102.128318, 0.15.2102.129448, 0.15.2103.133969, 0.15.2104.135508b1, 0.15.2104.136839b1, 0.16.2104.138035, 0.16.2105.140472, 0.16.2105.141895b1, 0.17.2105.143879, 0.17.2105.144881, 0.18.2106.148911, 0.18.2107.153439, 0.18.2107.154322a1, 0.18.2107.154378b1, 0.18.2107.154719a1, 0.18.2107.155031a1, 0.18.2108.31187a1, 0.18.2108.156779a1, 0.18.2108.1572

## Connecting to the Azure Quantum service

To connect to the Azure Quantum service, find the resource ID and location of your Workspace from the Azure Quantum portal here: https://portal.azure.com. Navigate to your Azure Quantum workspace and copy the values from the header.

<img src="..\copy_resource_id.png">

Paste the values into the `AzureQuantumProvider` constructor below to create a `provider` that connects to your Azure Quantum Workspace. Optionally, specify a default target:

In [6]:
from azure.quantum.cirq import AzureQuantumService
service = AzureQuantumService(
    resource_id="",
    location="",
    default_target="ionq.simulator"
)

### List all targets

You can now list all the targets that you have access to, including the current queue time and availability.

In [7]:
service.targets()

[<Target name="ionq.qpu", avg. queue time=345 s, Available>,
 <Target name="ionq.simulator", avg. queue time=4 s, Available>,
 <Target name="honeywell.hqs-lt-s1", avg. queue time=0 s, Unavailable>,
 <Target name="honeywell.hqs-lt-s1-apival", avg. queue time=0 s, Available>,
 <Target name="honeywell.hqs-lt-s2", avg. queue time=313169 s, Available>,
 <Target name="honeywell.hqs-lt-s2-apival", avg. queue time=0 s, Available>,
 <Target name="honeywell.hqs-lt-s1-sim", avg. queue time=1062 s, Available>]

## Run a simple circuit

Let's create a simple Cirq circuit to run. This circuit uses the square root of X gate, native to the IonQ hardware system.

In [8]:
import cirq

q0, q1 = cirq.LineQubit.range(2)
circuit = cirq.Circuit(
    cirq.X(q0)**0.5,             # Square root of X
    cirq.CX(q0, q1),              # CNOT
    cirq.measure(q0, q1, key='b') # Measure both qubits
)
circuit

0: ───X^0.5───@───M('b')───
              │   │
1: ───────────X───M────────

You can now run the program via the Azure Quantum service and get the result. The following cell will submit a job that runs the circuit with 100 shots, wait until the job is completed and return the results.

In [9]:
%%time
result = service.run(program=circuit, repetitions=100)

CPU times: user 74.9 ms, sys: 0 ns, total: 74.9 ms
Wall time: 12.5 s


This returns a `cirq.Result` object.

In [10]:
print(result)

b=1001100101100001000011011101000011010100010111100011001000100100010000001110010010101110110000011010, 1001100101100001000011011101000011010100010111100011001000100100010000001110010010101110110000011010


The previous job ran on the default simulator we specified, `"ionq.simulator"`. To run on the QPU, provide `"ionq.qpu"` as the `target` argument:

In [8]:
%%time
result = service.run(
    program=circuit,
    repetitions=100,
    target="ionq.qpu",
    timeout_seconds=500 # Set timeout to 500 seconds to accomodate current queue time on QPU
)

Again, this returns a `cirq.Result` object.

In [12]:
print(result)

b=0101011011011111100001011101101011011110100010000000011110111000100100110110101100110001001111101111, 0101011011011111100001011101101011011110100010000000011110111000100100110110101100110001001111101111


## Asynchronous model using Jobs

For long-running circuits, it can be useful to run them asynchronously. The `service.create_job` method returns a `Job`, which you can use to get the results after the job has run successfully.

In [11]:
%%time
job = service.create_job(
    program=circuit,
    repetitions=100,
    target="ionq.simulator"
)

CPU times: user 20.3 ms, sys: 12 ms, total: 32.3 ms
Wall time: 961 ms


To check on the job status, use `job.status()`:

In [12]:
job.status()

'completed'

To wait for the job to be done and get the results, use the blocking call `job.results()`:

In [13]:
%%time
result = job.results()
print(result)

00: 0.5
11: 0.5
CPU times: user 276 µs, sys: 143 µs, total: 419 µs
Wall time: 314 µs


Note that this does not return a `cirq.Result` object. Instead it returns a result object that is specific to the IonQ simulator and uses state probabilities instead of shot data.

In [14]:
type(result)

cirq_ionq.results.SimulatorResult

To convert this to a `cirq.Result` object, use `result.to_cirq_result()`:

In [15]:
result.to_cirq_result()

b=1110101111111110111000011101011111001100010000001011011101001111001111001101100111010000001100011100, 1110101111111110111000011101011111001100010000001011011101001111001111001101100111010000001100011100